In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [7]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv", index_col="PassengerId")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv", index_col="PassengerId")
soln = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

print(soln)
print(soln['PassengerId'][1])
print(soln['Transported'][1])
print(soln.iloc[1,1])
print(soln.iloc[1,0])

print(train_data)
print(test_data)
print(train_data.info())
print(test_data.info())
train_data['CryoSleep'] = train_data['CryoSleep'].astype(float)
train_data['VIP'] = train_data['VIP'].astype(float)
train_data['Transported'] = train_data['Transported'].astype(int)
print("-----------------------------------------------------")
print(train_data)

     PassengerId  Transported
0        0013_01        False
1        0018_01        False
2        0019_01        False
3        0021_01        False
4        0023_01        False
...          ...          ...
4272     9266_02        False
4273     9269_01        False
4274     9271_01        False
4275     9273_01        False
4276     9277_01        False

[4277 rows x 2 columns]
0018_01
False
False
0018_01
            HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
PassengerId                                                              
0001_01         Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
0002_01          Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
0003_01         Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
0003_02         Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
0004_01          Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...                ...       ...       ...            ...   ...    ..

In [8]:
from sklearn.model_selection import train_test_split
train_data.dropna(axis=0, subset=['Transported'], inplace=True)
y = train_data.Transported
train_data.drop(['Transported'], axis=1, inplace=True)
X = train_data
print(X)
print(y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
print(y_valid)

            HomePlanet  CryoSleep     Cabin    Destination   Age  VIP  \
PassengerId                                                             
0001_01         Europa        0.0     B/0/P    TRAPPIST-1e  39.0  0.0   
0002_01          Earth        0.0     F/0/S    TRAPPIST-1e  24.0  0.0   
0003_01         Europa        0.0     A/0/S    TRAPPIST-1e  58.0  1.0   
0003_02         Europa        0.0     A/0/S    TRAPPIST-1e  33.0  0.0   
0004_01          Earth        0.0     F/1/S    TRAPPIST-1e  16.0  0.0   
...                ...        ...       ...            ...   ...  ...   
9276_01         Europa        0.0    A/98/P    55 Cancri e  41.0  1.0   
9278_01          Earth        1.0  G/1499/S  PSO J318.5-22  18.0  0.0   
9279_01          Earth        0.0  G/1500/S    TRAPPIST-1e  26.0  0.0   
9280_01         Europa        0.0   E/608/S    55 Cancri e  32.0  0.0   
9280_02         Europa        0.0   E/608/S    TRAPPIST-1e  44.0  0.0   

             RoomService  FoodCourt  ShoppingMall 

In [9]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

col = [i for i in X_train.columns if X_train[i].dtype == "object"]
print(col)
print([X_train[i].value_counts() for i in col])

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
imputer = SimpleImputer()

#imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train[col]))
#imputed_X_train.columns = X_train.columns

#print(pd.get_dummies(X_train.HomePlanet))  #Hot one encoding only

encoder_X_train = pd.DataFrame(encoder.fit_transform(X_train[col]))
encoder_X_valid = pd.DataFrame(encoder.transform(X_valid[col]))
encoder_X_test = pd.DataFrame(encoder.transform(test_data[col]))

encoder_X_train.index = X_train.index
encoder_X_valid.index = X_valid.index
encoder_X_test.index = test_data.index
print(encoder_X_train.columns)
print(encoder_X_train)
print("--------------------------------------------------------------------------------------------")
print(encoder_X_valid.columns)
print(encoder_X_valid)
print("-----------------------------------------------------------------------------------")
print(encoder_X_test.columns)
print(encoder_X_test)


['HomePlanet', 'Cabin', 'Destination', 'Name']
[HomePlanet
Earth     3651
Europa    1708
Mars      1439
Name: count, dtype: int64, Cabin
E/13/S      7
F/1411/P    7
B/11/S      7
G/981/S     6
A/67/P      6
           ..
E/287/P     1
F/1270/S    1
G/920/P     1
F/702/P     1
G/474/P     1
Name: count, Length: 5449, dtype: int64, Destination
TRAPPIST-1e      4751
55 Cancri e      1432
PSO J318.5-22     622
Name: count, dtype: int64, Name
Troya Schwardson      2
Cuses Pread           2
Alraium Disivering    2
Keitha Josey          2
Anton Woody           2
                     ..
Joanry Welles         1
All Pittler           1
Natha Llovers         1
Ebonyd Swanardy       1
Cary Mejiaddox        1
Name: count, Length: 6787, dtype: int64]


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


RangeIndex(start=0, stop=12246, step=1)
             0      1      2      3      4      5      6      7      8      \
PassengerId                                                                  
4558_01        0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6326_01        1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0503_02        0.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0   
4757_01        1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9046_01        0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...            ...    ...    ...    ...    ...    ...    ...    ...    ...   
4655_01        0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
8423_01        1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5185_01        0.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0   
3499_04        1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2930_01        1.0    0.

In [14]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(n_estimators=500, learning_rate = 0.1)
model.fit(encoder_X_train,y_train)
prediction = model.predict(encoder_X_valid)
print(prediction)

perfection = 0
for i in range(len(prediction)):
    if prediction[i]>0.5:
        prediction[i] = 1
        if y_valid[i] == 1:
            perfection += 1
    else:
        prediction[i] = 0
        if y_valid[i] == 0:
            perfection += 1
print(perfection)
print(perfection/(len(prediction)+1))

actual_prediction = model.predict(encoder_X_test)
print(actual_prediction.shape)
print(soln.shape)
print(actual_prediction)
print(soln)

actual_prediction_df = pd.DataFrame(actual_prediction, columns=['Transported'])
print(actual_prediction_df)

perfection_actual = 0
for i in range(len(actual_prediction)):
    if actual_prediction[i]>0.5:
        actual_prediction_df.iloc[i,0] = True
        if soln.iloc[i,1] == True:
            perfection_actual += 1
    else:
        actual_prediction_df.iloc[i,0] = False
        if soln.iloc[i,1] == False:
            perfection_actual += 1
            
print(perfection_actual)
print(perfection_actual/(len(actual_prediction)+1))

dummy = pd.DataFrame(soln.iloc[:,0], columns=['PassengerId'])
original = dummy.join(actual_prediction_df)
print(original)
original.to_csv("submission.csv",index=None)
pd.read_csv("submission.csv")

[0.5007386  0.41674924 0.38726872 ... 0.5007386  0.6336531  0.6613698 ]
1017
0.5844827586206897
(4277,)
(4277, 2)
[0.38726872 0.38726872 0.7352056  ... 0.61252856 0.7109765  0.49646372]
     PassengerId  Transported
0        0013_01        False
1        0018_01        False
2        0019_01        False
3        0021_01        False
4        0023_01        False
...          ...          ...
4272     9266_02        False
4273     9269_01        False
4274     9271_01        False
4275     9273_01        False
4276     9277_01        False

[4277 rows x 2 columns]
      Transported
0        0.387269
1        0.387269
2        0.735206
3        0.661370
4        0.387269
...           ...
4272     0.387269
4273     0.387269
4274     0.612529
4275     0.710976
4276     0.496464

[4277 rows x 1 columns]
2831
0.6617578307620383
     PassengerId Transported
0        0013_01       False
1        0018_01       False
2        0019_01        True
3        0021_01        True
4        0023_01   

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [11]:
!wget kaggle competitions download -c spaceship-titanic

--2023-12-04 18:07:17--  http://kaggle/
Resolving kaggle (kaggle)... failed: Name or service not known.
wget: unable to resolve host address ‘kaggle’
--2023-12-04 18:07:17--  http://competitions/
Resolving competitions (competitions)... failed: Name or service not known.
wget: unable to resolve host address ‘competitions’
--2023-12-04 18:07:17--  http://download/
Resolving download (download)... failed: No address associated with hostname.
wget: unable to resolve host address ‘download’
--2023-12-04 18:07:17--  http://spaceship-titanic/
Resolving spaceship-titanic (spaceship-titanic)... failed: Name or service not known.
wget: unable to resolve host address ‘spaceship-titanic’
